# Parte 1 — Investigación: Conceptos Fundamentales de Prefect

## 1.1 Tasks en Prefect

### 1. ¿Qué es una Task en Prefect?
Una **Task** es una unidad de trabajo dentro de un pipeline. Básicamente es una función que hace algo específico (extraer datos, transformar, validar, guardar, etc.) y Prefect la puede monitorear, reintentar si falla y registrar su estado.  
La usamos cuando queremos dividir el pipeline en pasos claros y reutilizables.

### 2. ¿Qué significa que las Tasks sean “lazily evaluated”?  
Significa que **no se ejecutan cuando las llamás en el código**, sino recién cuando corre el *flow*.  
Prefect primero arma el grafo de dependencias (el DAG) y después ejecuta todo en el orden correcto.  
Esto permite que el pipeline sea más eficiente y flexible.

### 3. ¿Qué son los Task States?  
Son los **estados** que indican en qué etapa está una Task durante su ejecución.

| Estado      | ¿Cuándo ocurre? |
|-------------|------------------|
| **PENDING** | La task está lista, pero todavía no empezó a ejecutarse. |
| **RUNNING** | La task se está ejecutando en ese momento. |
| **COMPLETED** | Terminó bien sin errores. |
| **FAILED** | Falló durante la ejecución. |
| **RETRYING** | Falló pero tiene reintentos configurados y volverá a intentarse. |

## 1.2 Flows en Prefect

### 1. ¿Cuál es la diferencia entre un Flow y una Task? ¿Por qué necesitamos ambos?

Un **Flow** es el “pipeline completo”: coordina y ordena todos los pasos.  
Una **Task** es un paso individual dentro de ese pipeline.

Necesitamos ambos porque:
- Las Tasks dividen el trabajo en partes pequeñas y reutilizables.
- El Flow decide el orden, manejo de dependencias y ejecución general.

### 2. ¿Qué es un "subflow"? ¿Cuándo sería útil usar subflows?

Un **subflow** es un Flow que se llama desde dentro de otro Flow.  
Sirve para modularizar el pipeline, reutilizar partes completas o separar lógicas que se repiten (por ejemplo: `extract_and_validate()`).

### 3. ¿Cómo maneja Prefect las dependencias entre tasks? (DAG implícito)

Prefect crea el DAG **automáticamente**:  
cuando una Task usa el resultado de otra (`transform(extract())`), Prefect detecta esa dependencia sin que la declares explícitamente.

Así arma un **DAG implícito**, ejecutando cada Task en el orden correcto según lo que necesita.


## 1.3 Investigación avanzada: Results y Caching

### 1. ¿Qué es el "result persistence"? ¿Por qué es importante en pipelines de datos?

Es la capacidad de Prefect de **guardar y recuperar resultados de tasks** (por ejemplo en archivos, S3, bases de datos o memoria).

Es importante porque:
- Evita recalcular pasos costosos
- Permite reanudar un pipeline que falló
- Facilita debugging y auditoría
- Mejora reproducibilidad

### 2. ¿Cómo funciona el caching en Prefect? ¿Qué parámetro usarían para cachear el resultado de una task?

Prefect evita ejecutar una task si ya tiene un **resultado válido en caché** generado con los mismos parámetros.

Se habilita usando:
- `cache_expiration` (tiempo de validez)
- **o** `cache_key_fn` (clave de caché personalizada)

### 3. ¿Qué es una `cache_key_fn`? Den un ejemplo de cuándo la usarían.

Es una función que genera **manual y explícitamente** la clave de caché de una task.  
Sirve cuando no querés que el caché dependa de *todos* los parámetros, sino solo de algunos.

**Ejemplo:**  
Cachear una extracción diaria usando solo la fecha:

```python
def key_fn(_, date):
    return f"extract-{date}"
```

Así, si ya extrajiste los datos del día 2025-10-01, no se vuelve a ejecutar la tarea.

# Parte 2 — Diseño Conceptual

## 2.1 Arquitectura del escenario

**Escenario elegido:** Transacciones diarias de un e-commerce.

| Rol                 | ¿Quién sería en su escenario? |
|--------------------|-------------------------------|
| Business data owner | Equipo de ventas del e-commerce |
| Data engineers      | Equipo de ingeniería de datos que mantiene el pipeline ETL |
| Data consumers      | Analistas de negocio y dashboards de reporting |

## 2.2 Tipo de pipeline

- **Tipo elegido (batch/streaming):** Batch  
- **Justificación:**  
  Las ventas se procesan de manera diaria. No necesitamos procesamiento en tiempo real, y el enfoque batch es más eficiente para análisis agregados, validación de calidad de datos y reintentos controlados. Además, simplifica el diseño del ETL y reduce costos operativos.


# Parte 3 — Implementación del Pipeline Base

In [ ]:
# 3.1 Setup

# Instalación de Prefect
!pip install -q prefect pandas

# Importar librerías
from prefect import flow, task
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Entorno configurado correctamente")
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✅ Entorno configurado correctamente
📅 Fecha: 2025-12-02 21:31


In [ ]:
# 3.2 Implementar Tasks

# === TASK 1: EXTRACT ===
# Usamos tags y log_prints para que todo quede registrado en Prefect

@task(tags=["extract"], log_prints=True)
def extract_data():
    """
    Extrae datos de la fuente.
    """
    np.random.seed(42)
    n_rows = 100

    data = {
        'fecha': pd.date_range(start='2024-01-01', periods=n_rows, freq='D'),
        'producto': np.random.choice(['A', 'B', 'C', 'D'], n_rows),
        'cantidad': np.random.randint(1, 50, n_rows),
        'precio_unitario': np.random.uniform(10, 100, n_rows).round(2),
        'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], n_rows)
    }

    df = pd.DataFrame(data)
    print(f"📥 Extraídos {len(df)} registros")
    return df


# === TASK 2: TRANSFORM ===
@task(tags=["transform"], log_prints=True)
def transform_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica transformaciones a los datos.
    """
    df['total'] = df['cantidad'] * df['precio_unitario']

    df['ticket_size'] = pd.cut(
        df['total'],
        bins=[0, 100, 500, float('inf')],
        labels=['small', 'medium', 'large']
    )

    print(f"🔄 Transformados {len(df)} registros")
    return df


# === TASK 3: LOAD ===
@task(tags=["load"], log_prints=True, retries=2, retry_delay_seconds=3)
def load_data(df: pd.DataFrame, output_path: str = "output.csv"):
    """
    Carga los datos al destino final.
    """
    df.to_csv(output_path, index=False)
    print(f"💾 Guardados {len(df)} registros en {output_path}")


In [5]:
# 3.3 Implementar Flow
 
# === FLOW: Orquestador del pipeline ===

@flow(name="ETL Pipeline Ventas", log_prints=True)
def etl_flow():
    """
    Flow principal que orquesta las tasks ETL.
    """
    df_raw = extract_data()
    df_clean = transform_data(df_raw)
    load_data(df_clean)

    print("\n✅ Pipeline ETL completado exitosamente!")
    return df_clean


# === EJECUTAR ===
if __name__ == "__main__":
    resultado = etl_flow()


21:35:24.571 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8364
See https://docs.prefect.io/v3/concepts/server#how-to-guides for more information on running a dedicated Prefect server.

21:35:26.206 | INFO    | Flow run 'intelligent-swift' - Beginning flow run 'intelligent-swift' for flow 'ETL Pipeline Ventas'

21:35:26.273 | INFO    | Task run 'extract_data-daa' - 📥 Extraídos 100 registros

21:35:26.293 | INFO    | Task run 'extract_data-daa' - Finished in state Completed()

21:35:26.372 | INFO    | Task run 'transform_data-c9c' - 🔄 Transformados 100 registros

21:35:26.390 | INFO    | Task run 'transform_data-c9c' - Finished in state Completed()

21:35:26.428 | INFO    | Task run 'load_data-49f' - 💾 Guardados 100 registros en output.csv

21:35:26.446 | INFO    | Task run 'load_data-49f' - Finished in state Completed()

21:35:26.447 | INFO    | Flow run 'intelligent-swift' - 
✅ Pipeline ETL completado exitosamente!

21:35:26.453 | INFO    | Flow run 'intelligent-swift' - Finished in state Completed()

## 3.4 Preguntas de observación

### 1. ¿Qué información muestra Prefect en los logs? Copien un fragmento y explíquenlo.

**Fragmento real de mis logs:**

INFO | Flow run 'intelligent-swift' - Beginning flow run 'ETL Pipeline Ventas'
INFO | Task run 'extract_data-daa' - 📥 Extraídos 100 registros
INFO | Task run 'extract_data-daa' - Finished in state Completed()
INFO | Task run 'transform_data-c9c' - 🔄 Transformados 100 registros
INFO | Task run 'transform_data-c9c' - Finished in state Completed()
INFO | Task run 'load_data-49f' - 💾 Guardados 100 registros en output.csv
INFO | Task run 'load_data-49f' - Finished in state Completed()
INFO | Flow run 'intelligent-swift' - Finished in state Completed()


**Explicación:**  
Los logs muestran cada paso del pipeline: el inicio del flow, la ejecución de cada task, los prints internos y el estado final de cada una (`Completed`). También muestra cuántos registros se procesaron y cuándo finaliza todo el ETL. Es información clave para monitorear el pipeline y detectar errores.

### 2. ¿En qué orden se ejecutaron las tasks? ¿Cómo lo infiere Prefect?

El orden fue:

1. `extract_data`
2. `transform_data`
3. `load_data`

Prefect lo infiere automáticamente porque construye un **DAG implícito**:  
cada task recibe como parámetro la salida de la anterior, por lo que entiende que esas dependencias deben respetarse y ejecuta en ese orden sin que uno lo tenga que especificar manualmente.

### 3. ¿Qué pasaría si una task falla? ¿Qué estados tendría el flow?

Si una task falla:

- La task entra en estado **FAILED** o **RETRYING** si tiene reintentos.
- Todas las tasks que dependan de ella no se ejecutan.
- El flow completo termina en **FAILED** o **CRASHED**, según el tipo de error.

Prefect detiene automáticamente el pipeline para evitar resultados inconsistentes.


# Parte 4 — Investigación: Funcionalidades Avanzadas

## 4.1 Retries y manejo de errores

### 1. ¿Qué parámetros controlan los retries?

| Parámetro | Descripción | Valor por defecto |
|----------|-------------|-------------------|
| `retries` | Número de veces que se intentará repetir la task si falla. | `0` |
| `retry_delay_seconds` | Tiempo fijo entre reintentos. | `0` |
| `retry_jitter_factor` | Factor que agrega aleatoriedad al delay para evitar colisiones. | `0` |

### 2. ¿Qué es exponential backoff?

Exponential backoff es una estrategia donde cada reintento espera el doble que el anterior  
(1s → 2s → 4s → 8s…).  
Sirve para no saturar APIs/servidores cuando hay fallos repetidos.

En Prefect se puede lograr usando `retry_delay_seconds` junto con `retry_jitter_factor`,  
o implementando un delay creciente dentro de la lógica de la task.

In [6]:
@task(retries=3, retry_delay_seconds=2, log_prints=True)
def extract_data_with_retry():
    """Task con reintentos automáticos."""
    # Simular fallo aleatorio para probar retries
    if np.random.random() < 0.5:
        raise Exception("Error simulado de conexión")
    return "datos extraídos"

In [7]:
@flow
def test_retry_flow():
    return extract_data_with_retry()

test_retry_flow()

21:47:59.421 | INFO    | Flow run 'rare-honeybee' - Beginning flow run 'rare-honeybee' for flow 'test-retry-flow'

21:47:59.447 | INFO    | Task run 'extract_data_with_retry-b4b' - Task run failed with exception: Exception('Error simulado de conexión') - Retry 1/3 will start 2 second(s) from now

21:48:01.454 | INFO    | Task run 'extract_data_with_retry-b4b' - Finished in state Completed()

21:48:01.471 | INFO    | Flow run 'rare-honeybee' - Finished in state Completed()

'datos extraídos'

## 4.2 Caching de resultados

### 1. ¿Qué es `cache_expiration`? ¿Cómo se especifica?

`cache_expiration` define cuánto tiempo es válido el resultado cacheado de una task.
Si la task corre nuevamente dentro de ese tiempo, Prefect no la re-ejecuta y devuelve el resultado almacenado.

Se especifica usando un `timedelta`, por ejemplo:

```python
@task(cache_expiration=timedelta(minutes=10))
```

### 2. ¿Cuándo es útil cachear una task? Den 2 ejemplos de su escenario.

**Ejemplo 1:**
Cuando la extracción de datos es costosa (por ejemplo, acceder a una API lenta o pagada).

**Ejemplo 2:**
Cuando los datos no cambian todo el tiempo (por ejemplo, ventas del día que solo se actualizan una vez por día).

### 3. ¿Qué pasa si los inputs de la task cambian? ¿Se usa el cache?

**No.**
Si los inputs de la task cambian, Prefect genera una clave de caché distinta y **vuelve a ejecutar la task.**
El cache solo se usa cuando los parámetros de entrada y el tiempo de expiración coinciden.

In [8]:
from datetime import timedelta

# Implementación del caching en la task de extracción
@task(cache_expiration=timedelta(minutes=5), log_prints=True)
def extract_data_cached():
    """Task con caching — no re-ejecuta si ya corrió recientemente."""
    print("⏳ Ejecutando extracción (esto NO debería aparecer si está cacheado)")
    return extract_data()

In [11]:
@flow
def test_cache_flow():
    return extract_data_cached()

test_cache_flow()
test_cache_flow()   # segunda ejecución → debería evitar la extracción


22:01:29.473 | INFO    | Flow run 'versatile-goldfish' - Beginning flow run 'versatile-goldfish' for flow 'test-cache-flow'

22:01:29.493 | INFO    | Task run 'extract_data_cached-19a' - ⏳ Ejecutando extracción (esto NO debería aparecer si está cacheado)

22:01:29.516 | INFO    | Task run 'extract_data-4b1' - 📥 Extraídos 100 registros

22:01:29.535 | INFO    | Task run 'extract_data-4b1' - Finished in state Completed()

22:01:29.536 | INFO    | Task run 'extract_data_cached-19a' - Finished in state Completed()

...

22:01:29.617 | INFO    | Task run 'extract_data-a59' - 📥 Extraídos 100 registros

22:01:29.635 | INFO    | Task run 'extract_data-a59' - Finished in state Completed()

22:01:29.637 | INFO    | Task run 'extract_data_cached-ebc' - Finished in state Completed()

22:01:29.644 | INFO    | Flow run 'ruddy-urchin' - Finished in state Completed()

,fecha,producto,cantidad,precio_unitario,region
0,2024-01-01,C,18,93.67,Oeste
1,2024-01-02,D,26,82.73,Norte
2,2024-01-03,A,44,67.01,Norte
3,2024-01-04,C,34,88.43,Oeste
4,2024-01-05,C,10,82.33,Norte
...,...,...,...,...,...
95,2024-04-05,B,13,69.72,Sur
96,2024-04-06,B,32,10.46,Oeste
97,2024-04-07,D,7,24.47,Este
98,2024-04-08,B,22,59.39,Sur


## 4.3 Logging personalizado

### 1. ¿Cómo se accede al logger de Prefect dentro de una task?

Usando la función:

```python
from prefect import get_run_logger
logger = get_run_logger()
```

### 2. ¿Qué niveles de log soporta Prefect? Listen al menos 4.

1. logger.debug()
2. logger.info()
3. logger.warning()
4. logger.error()

### 3. ¿Cómo configurarían el nivel de log para ver más detalle?

En el decorador de la task o del flow, usando:

```python
@flow(log_prints=True)
```

In [12]:
from prefect import get_run_logger

@task
def transform_data_with_logging(df: pd.DataFrame) -> pd.DataFrame:
    """Task con logging estructurado."""
    
    logger = get_run_logger()   # obtener el logger de Prefect

    logger.info(f"Iniciando transformación de {len(df)} registros")  # nivel info

    df['total'] = df['cantidad'] * df['precio_unitario']

    # Log de estadísticas
    logger.info(f"Total ventas: ${df['total'].sum():,.2f}")  # nivel info
    logger.info(f"Detalle por región: {df.groupby('region')['total'].sum().to_dict()}")

    return df


## 4.4 Concurrencia y paralelismo

### 1. ¿Qué es un Task Runner? ¿Cuál es el default?

Un **Task Runner** es el componente que define cómo se ejecutan las tasks dentro de un flow:
- secuencialmente,
- en paralelo,
- con hilos,
- con procesos, etc.

El **default** en Prefect 2.x es: `ConcurrentTaskRunner`.

### 2. ¿Qué Task Runners ofrece Prefect? Describan al menos 2:

| Task Runner | ¿Cuándo usarlo? |
|----------|-------------|
| `ConcurrentTaskRunner` | Para ejecutar tasks en paralelo usando threads. Ideal para I/O (APIs, lectura/escritura). |
| `SequentialTaskRunner` | Para ejecutar tasks una por una. Útil para debugging o pipelines sin paralelismo. |

### 3. ¿Cómo ejecutarían tasks en paralelo? Investiguen .submit() y .map().

**.submit():**
Ejecuta una task asíncrona, devolviendo un Future.
Permite lanzar varias tasks a la vez sin esperar que terminen.

**.map():**
Ejecuta la misma task varias veces en paralelo, una por cada elemento de una lista.

In [13]:
from prefect import flow, task
from prefect.task_runners import ConcurrentTaskRunner

@task
def process_region(region: str, df: pd.DataFrame) -> dict:
    """Procesa datos de una región específica."""
    df_region = df[df['region'] == region]
    return {
        'region': region,
        'total': df_region['total'].sum(),
        'count': len(df_region)
    }

@flow(task_runner=ConcurrentTaskRunner())  # usar concurrencia real
def etl_flow_parallel():
    df_raw = extract_data()
    df_clean = transform_data(df_raw)

    # Procesar cada región en paralelo
    regiones = ['Norte', 'Sur', 'Este', 'Oeste']
    futures = [process_region.submit(r, df_clean) for r in regiones]  # método async

    # Esperar resultados
    results = [f.result() for f in futures]  # obtener resultados

    print(f"📊 Resultados por región: {results}")
    return results

# Parte 5 — Investigación: Deployments y Scheduling

## 5.1 Conceptos de Deployment

### 1. ¿Qué es un Deployment en Prefect? ¿Cuál es la diferencia entre un Flow y un Deployment?

Un **Deployment** es una *configuración ejecutable* de un Flow: incluye cómo, cuándo y dónde se ejecuta.
El **Flow** es el *código Python* que define la lógica del pipeline.
El **Deployment** es la *instalación* del flujo en Prefect, con parámetros como:

* scheduling
* work pool
* versión
* parámetros del flow
* infraestructura usada
* almacenamiento

En resumen:
*Flow = lógica*
*Deployment = ejecución programada y versionada del flow*

### 2. ¿Qué es un Work Pool? ¿Para qué sirve?

Un **Work Pool** es un grupo lógico que administra *qué infraestructura* se usará para ejecutar los deployments.
Sirve para:

* organizar distintos entornos (dev, staging, prod)
* conectar Workers con Flows
* administrar colas de ejecución
* separar cargas de trabajo por equipo o tipo de tarea

Es, básicamente, el “puente” entre los deployments y los workers.

### 3. ¿Qué es un Worker? ¿Cómo se relaciona con el Work Pool?

Un **Worker** es un proceso que:

* se registra en un Work Pool
* escucha tareas pendientes
* toma deployments y los ejecuta

La relación es:

*Deployment → Work Pool → Worker → Ejecuta el Flow*

El Deployment se envía al Work Pool, el Work Pool lo pone en cola, y los Workers conectados lo levantan y ejecutan.

## 5.2 Scheduling

### 1. ¿Qué tipos de schedules soporta Prefect?

Describan al menos 3:

| Tipo de Schedule     | Descripción                                                                             | Ejemplo                                               |
| -------------------- | --------------------------------------------------------------------------------------- | ----------------------------------------------------- |
| **CronSchedule**     | Usa sintaxis cron para ejecutar flows en horarios repetitivos.                          | `"0 6 * * *"` → todos los días a las 6 AM             |
| **IntervalSchedule** | Ejecuta el flow cada cierto intervalo fijo.                                             | `timedelta(hours=1)` → cada 1 hora                    |
| **RRuleSchedule**    | Usa reglas RFC 5545 para horarios complejos (frecuencias, excepciones, múltiples días). | “Cada lunes y miércoles a las 9 AM, excepto feriados” |


### 2. ¿Cómo expresarían "ejecutar todos los días a las 6 AM" en cron?

```cron
0 6 * * *
```

### 3. ¿Qué es un `RRuleSchedule`? ¿Cuándo lo usarían sobre cron?

`RRuleSchedule` implementa las reglas RFC 5545 (iCalendar), permitiendo programaciones **mucho más complejas** que cron:

* horarios irregulares
* múltiples días específicos
* intervalos combinados
* exclusiones (no correr feriados)
* reglas encadenadas

Lo usaría en lugar de cron cuando necesito lógica de calendario más avanzada que cron *no puede expresar fácilmente*, como:

> “Correr el pipeline sólo los días hábiles del mes, a las 9 AM, excluyendo feriados que se definan aparte.”


## 5.3 Crear un Deployment (conceptual)

In [15]:
# Completar basándose en la documentación de Deployments
# https://docs.prefect.io/latest/concepts/deployments/

# Opción 1: Usando serve() - más simple
# if __name__ == "__main__":
#     etl_flow.serve(
#         name="etl-diario-ecommerce",  # nombre del deployment
#         cron="0 6 * * *",  # schedule en formato cron
#         tags=["etl", "ecommerce"],  # tags para organización
#     )

# Opción 2: Usando deploy() - más control
etl_flow.deploy(
    name="etl-diario-ecommerce",
    work_pool_name="local-pool",
    cron="0 6 * * *",
)

<coroutine object sync_compatible.<locals>.coroutine_wrapper.<locals>.ctx_call at 0x1611609a0>

# Parte 6 — Extensión DataOps

## Opción A — Validación con logging estructurado

In [16]:
from prefect import task, get_run_logger

@task(
    retries=3,                   # La task se reintenta 3 veces si falla
    retry_delay_seconds=2        # Espera 2 segundos entre reintentos
)
def validate_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Valida la calidad de los datos usando logging estructurado.
    Incluye validaciones básicas y reporta errores antes de continuar.
    """

    logger = get_run_logger()
    errors = []

    # --- Inicio del proceso de validación ---
    logger.info("🔍 Iniciando validación de datos")

    # 1) Validación: DataFrame vacío
    if len(df) == 0:
        logger.error("❌ DataFrame vacío detectado")
        errors.append("DataFrame vacío")

    # 2) Validación: valores nulos por columna
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        logger.warning(f"⚠️ Valores nulos encontrados: {null_counts.to_dict()}")

    # 3) Validación: columnas esperadas
    columnas_esperadas = {"fecha", "producto", "cantidad", "precio_unitario", "region", "total"}
    columnas_actuales = set(df.columns)

    if columnas_esperadas - columnas_actuales:
        faltantes = columnas_esperadas - columnas_actuales
        logger.error(f"❌ Columnas faltantes: {faltantes}")
        errors.append(f"Columnas faltantes: {faltantes}")

    # Si hubo errores → cortar ejecución
    if errors:
        raise ValueError(f"Validación fallida: {errors}")

    # --- Fin de validación ---
    logger.info("✅ Validación exitosa")
    return df

**¿Qué hace esta task?**

La task `validate_data()` implementa reglas de calidad de datos y usa **logging estructurado de Prefect** para registrar:
* mensajes informativos (`logger.info`)
* advertencias (`logger.warning`)
* errores (`logger.error`)

Esto permite auditar qué pasó con los datos y facilita debugging.

**Se agregan:**

```python
retries=3
retry_delay_seconds=2
```

porque en pipelines reales los fallos suelen ser temporales (e.g., un archivo no llegó todavía).
Con estos parámetros la task:
* intenta 3 veces
* espera 2 segundos entre cada intento

**Validaciones implementadas**
1. DataFrame vacío
2. Valores nulos
3. Columnas faltantes
4. Si hay errores → se corta el flow

# Parte 7 — Reflexión y Conexión con DataOps (5 min)¶

## 7.1 Conceptos de Prefect

### 1. ¿Cómo ayuda Prefect a implementar el principio de "Observabilidad" de DataOps?

Prefect registra automáticamente cada etapa del pipeline: estados, logs, retries, tiempos de ejecución y errores.
Esto permite **ver qué pasó, dónde falló y por qué**, logrando trazabilidad completa sin agregar infraestructura extra.
En DataOps, esto es clave para detectar problemas rápido y medir la salud del pipeline.

### 2. ¿Cómo ayuda el caching a la "Reproducibilidad"?

El caching garantiza que, si los inputs no cambiaron, la task devuelve **exactamente el mismo resultado**.
Esto elimina variaciones accidentales, acelera re-ejecuciones y permite reconstruir outputs pasados de manera consistente.
En resumen: **mismos datos → mismo resultado**, siempre.

### 3. ¿Cómo conectan los Deployments con "CI/CD para datos"?

Un Deployment convierte un flow en un artefacto versionado, ejecutable y schedulable.
Esto permite integrarlo en pipelines de CI/CD: cuando se hace un commit, el deployment se actualiza, se testea y se despliega automáticamente.
Es la forma en que Prefect permite **automatizar, versionar y publicar** pipelines como si fueran software.

## 7.2 Comparación con alternativas

### 1. ¿Qué diferencias hay entre Prefect y Apache Airflow? Mencionen al menos 2.

Diferencia 1: Prefect usa un DAG **implícito**, construido según el orden del código; Airflow requiere declarar DAGs de forma explícita y más verbosa.

Diferencia 2: Prefect tiene **mejor manejo de estados, retries y logs** sin configuración adicional.
Airflow requiere más setup y es más rígido en su arquitectura.

### 2. ¿Qué es Dagster? ¿En qué se diferencia de Prefect?

Dagster es un framework de orquestación enfocado en data assets: permite definir datasets como entidades con dependencias explícitas.
Prefect, en cambio, se centra en **flows y tasks**, con mucha más flexibilidad y menos estructura obligatoria.